In [ ]:
!nvidia-smi

Sat Oct  8 19:31:13 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install datasets transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 432 kB 17.6 MB/s 
     |████████████████████████████████| 4.9 MB 55.9 MB/s 
     |████████████████████████████████| 212 kB 15.6 MB/s 
     |████████████████████████████████| 163 kB 55.3 MB/s 
     |████████████████████████████████| 115 kB 63.2 MB/s 
     |████████████████████████████████| 127 kB 45.7 MB/s 
     |████████████████████████████████| 6.6 MB 60.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
!cp drive/MyDrive/vtb_news_hack.zip ./

In [ ]:
!unzip vtb_news_hack.zip

Archive:  vtb_news_hack.zip
  inflating: vtb_news_hack.csv       


In [ ]:
import json
import re
from glob import glob
from pathlib import Path

import datasets
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MultiLabelBinarizer
import torch
import transformers
from datasets import load_dataset
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from transformers import (AutoTokenizer, AutoModelForSequenceClassification, 
                          TrainingArguments, Trainer)

ModuleNotFoundError: ignored

In [ ]:
df = pd.read_csv("vtb_news_hack.csv")
df = df[~df["summary"].isna()]

In [ ]:
df = df.drop_duplicates(subset=["summary"])
df["target"].value_counts()

0    12818
1      282
Name: target, dtype: int64

In [ ]:
df = df[df["summary"].str.len() >= 50]
df["target"].value_counts()

0    12229
1      277
Name: target, dtype: int64

In [ ]:
df.columns = ["text", "label"]

In [ ]:
df["text"] = df["text"].apply(lambda x: re.sub("<.*?>", "", x))

In [ ]:
df = df[df["text"].str.len() >= 50]
df["label"].value_counts()

0    12192
1      277
Name: label, dtype: int64

In [ ]:
df = df.drop_duplicates(subset=["text"])
df["label"].value_counts()

0    12175
1      277
Name: label, dtype: int64

In [ ]:
train, test = train_test_split(df, test_size=0.2, random_state=42)
val, test = train_test_split(test, test_size=0.5, random_state=42)

In [ ]:
df_dataset = {
    "train": train,
    "val": val,
    "test": test,
}

In [ ]:
from datasets import Dataset

def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        max_length=256,
        padding='max_length',
        truncation=True
    )

# model_ckpt = "bert-base-multilingual-uncased"
model_ckpt = "cointegrated/rubert-tiny2"
tokenizer = AutoTokenizer.from_pretrained(
    model_ckpt, problem_type="multi_label_classification")

hf_datasets = dict()
for key, value in df_dataset.items():
    dataset = Dataset.from_pandas(value, split="key")
    hf_datasets[key] = dataset.map(tokenize_function, batched=True)

Downloading:   0%|          | 0.00/401 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.74M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

  0%|          | 0/10 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [ ]:
num_labels = df_dataset["train"]["label"].unique().shape[0]
model = AutoModelForSequenceClassification.from_pretrained(
    model_ckpt,
    num_labels=num_labels,
    problem_type="single_label_classification").to('cuda')

Downloading:   0%|          | 0.00/715 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/118M [00:00<?, ?B/s]

Some weights of the model checkpoint at cointegrated/rubert-tiny2 were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not 

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # print("pred", predictions)
    # print("label", labels)
    pred_labels = np.argmax(predictions, axis=1)
    f1_results = f1_score(labels,
                          pred_labels,
                          # (predictions > 0.5).astype(int),
                          average=None)
    f1_micro = f1_score(labels,
                        pred_labels,
                        average="micro")
    f1_macro = f1_score(labels,
                        pred_labels,
                        average="macro")
    output = {
        # 'accuracy_thresh': accuracy_thresh(predictions, labels),
        # 'f1': f1_results,
        "f1_micro": f1_micro,
        "f1_macro": f1_macro
        }
    # for k, v in label_dict.items():
    #     output[f"f1_{k}"] = f1_results[v]
    return output

In [ ]:
from transformers import EarlyStoppingCallback
batch_size = 256

args = TrainingArguments(
    output_dir="multiclass_model",
    save_strategy = "epoch",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=10,
    weight_decay=0.01,
    save_total_limit=5,
   load_best_model_at_end=True
)


trainer = Trainer(
    model,
    args,
    train_dataset=hf_datasets["train"],
    eval_dataset=hf_datasets["val"],
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    callbacks=[EarlyStoppingCallback(4)],
)

trainer.evaluate()
trainer.train()

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: __index_level_0__, text. If __index_level_0__, text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1245
  Batch size = 256
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: __index_level_0__, text. If __index_level_0__, text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 9961
  Num Epochs = 10
  Instantaneous batch size per device = 256
  Total train batch size (w. parallel, distributed & accumulation) = 256
  Gradient Accumulation steps = 1
  Total optimization steps = 390


Epoch,Training Loss,Validation Loss,F1 Micro,F1 Macro
1,No log,0.174044,0.983936,0.495951
2,No log,0.085176,0.983936,0.495951
3,No log,0.076971,0.983936,0.495951
4,No log,0.064691,0.983936,0.495951
5,No log,0.047585,0.984739,0.543771
6,No log,0.037926,0.987149,0.711036
7,No log,0.032046,0.991165,0.840617
8,No log,0.029612,0.989558,0.821675
9,No log,0.028058,0.989558,0.830681
10,No log,0.027672,0.989558,0.830681


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: __index_level_0__, text. If __index_level_0__, text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1245
  Batch size = 256
Saving model checkpoint to multiclass_model/checkpoint-39
Configuration saved in multiclass_model/checkpoint-39/config.json
Model weights saved in multiclass_model/checkpoint-39/pytorch_model.bin
tokenizer config file saved in multiclass_model/checkpoint-39/tokenizer_config.json
Special tokens file saved in multiclass_model/checkpoint-39/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: __index_level_0__, text. If __index_level_0__, text are not expected by `BertForSequenceClassification.forward`,  you can 

TrainOutput(global_step=390, training_loss=0.10183184697077824, metrics={'train_runtime': 306.529, 'train_samples_per_second': 324.961, 'train_steps_per_second': 1.272, 'total_flos': 367272378439680.0, 'train_loss': 0.10183184697077824, 'epoch': 10.0})

In [ ]:
trainer.save_model("vtb_model2")

Saving model checkpoint to vtb_model2
Configuration saved in vtb_model2/config.json
Model weights saved in vtb_model2/pytorch_model.bin
tokenizer config file saved in vtb_model2/tokenizer_config.json
Special tokens file saved in vtb_model2/special_tokens_map.json


In [ ]:
!zip vtb_model2.zip vtb_model2/*

  adding: vtb_model2/config.json (deflated 49%)
  adding: vtb_model2/pytorch_model.bin (deflated 8%)
  adding: vtb_model2/special_tokens_map.json (deflated 42%)
  adding: vtb_model2/tokenizer_config.json (deflated 43%)
  adding: vtb_model2/tokenizer.json (deflated 73%)
  adding: vtb_model2/training_args.bin (deflated 48%)
  adding: vtb_model2/vocab.txt (deflated 64%)


In [ ]:
cp vtb_model2.zip drive/MyDrive/

In [ ]:
ls -lah drive/MyDrive/vtb*

-rw------- 1 root root 104M Oct  8 09:54 drive/MyDrive/vtb_model2.zip
-rw------- 1 root root 104M Oct  8 09:19 drive/MyDrive/vtb_model.zip
-rw------- 1 root root 5.4M Oct  7 22:07 drive/MyDrive/vtb_news_hack.zip


In [ ]:
from transformers import pipeline

In [ ]:
!cp drive/MyDrive/vtb_model2.zip ./

In [ ]:
!unzip vtb_model2.zip

Archive:  vtb_model.zip
  inflating: vtb_model/config.json   
  inflating: vtb_model/pytorch_model.bin  
  inflating: vtb_model/special_tokens_map.json  
  inflating: vtb_model/tokenizer_config.json  
  inflating: vtb_model/tokenizer.json  
  inflating: vtb_model/training_args.bin  
  inflating: vtb_model/vocab.txt     


In [ ]:
pipe = pipeline("text-classification", "vtb_model2", device=0)

loading configuration file vtb_model2/config.json
Model config BertConfig {
  "_name_or_path": "vtb_model2",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "emb_size": 312,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 312,
  "initializer_range": 0.02,
  "intermediate_size": 600,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 2048,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 3,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.22.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 83828
}

loading configuration file vtb_model2/config.json
Model config BertConfig {
  "_name_or_path": "vtb_model2",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_pro

In [ ]:
pipe("Украина")

[{'label': 'LABEL_0', 'score': 0.9890224933624268}]

In [ ]:
pipe("бухгалтер")

[{'label': 'LABEL_0', 'score': 0.6556780934333801}]

In [ ]:
tokenizer_kwargs = {'padding':True,'truncation':True,'max_length':512}

for i, text in enumerate(df[df["label"] == 0]["text"].values):
    label = pipe(text, **tokenizer_kwargs)[0]["label"]
    if label == "LABEL_1":
        print(text)
        print("-" * 80)

Минфин посчитал расходы по налоговым льготам с прогнозом до 2025 года. Ранее там предупреждали, что исходя из эффективности налоговых льгот их предоставление может быть ограничено или прекращено.
--------------------------------------------------------------------------------
Во время льготного периода можно вносить посильные платежи по кредиту.
--------------------------------------------------------------------------------
Считать объем мусора, отправленного на помойку, будут по иным нормативам. Но плата за вывоз отходов от этого существенно не изменится.
--------------------------------------------------------------------------------
Если вы купили квартиру или оплатили медицинские услуги, вы имеете право вернуть часть денег с помощью налогового вычета.
--------------------------------------------------------------------------------
Как составить договор задатка при покупке квартиры по образцу 2021 года? - в статье рассказываем о договоре задатка, как его составить, а также как не д

In [ ]:
df.iloc[1202]

text     Публикуем разъяснения юриста.
label                                0
Name: 1543, dtype: object

In [ ]:
tokenizer_kwargs = {'padding':True,'truncation':True,'max_length':512}

for i, text in enumerate(df[df["label"] == 0]["text"].values):
    label = pipe(text, **tokenizer_kwargs)[0]["label"]
    if label == "LABEL_1":
        print(i, text)

1144 Во время льготного периода можно вносить посильные платежи по кредиту.
1190 Дополнительное обучение, которое поможет прокачать предпринимательские компетенции, смогут в этом году пройти в 47 вузах 30 тысяч нынешних студентов.
1202 Публикуем разъяснения юриста.
1243 Достаточно загранпаспорта, но лучше дополнительно взять военный билет, справки из военкомата, с места работы или учебы.
1360 Треть малого бизнеса сталкивается с проблемой оплаты своей работы крупными компаниями. Но рычагов воздействия на таких контрагентов почти нет.
2265 Как составить договор задатка при покупке квартиры по образцу 2021 года? - в статье рассказываем о договоре задатка, как его составить, а также как не дать обмануть себя! Читайте в нашей статье
3402 Судебные приставы оштрафовали коллекторское агентство за звонки гражданину, признанному банкротом. Это крайне важное дело: ни один коллектор не вправе тревожить бывших должников. Иначе сам окажется в должниках.
3615 С 2011 года вступил в силу федеральный за

KeyboardInterrupt: ignored

In [ ]:
preds = []

for text in df_dataset["val"]["text"].values:
    preds.append(pipe(text, **tokenizer_kwargs))

In [ ]:
preds[0]

In [ ]:
processed_preds = [int(p[0]["label"][6:]) for p in preds]
processed_proba = [p[0]["score"] for p in preds]

In [ ]:
def check_metrics(  true_y,
                    pred_y,
                    pred_y_proba,
                    threshold):
        assert len(true_y) == len(pred_y)

        bad_answers = []
        accuracy = []

        for i in range(len(true_y)):

            if pred_y[i] != true_y[i] and pred_y_proba[i] > threshold:
                # F sure(FP)
                bad_answers.append(1)

            if pred_y[i] == true_y[i] and pred_y_proba[i] > threshold:
                accuracy.append(1)

        return (np.around([sum(accuracy) / len(true_y) * 100], 5)[0],
                np.around(sum(bad_answers) / len(true_y) * 100, 5)
                )

In [ ]:
for threshold in (0.4, 0.5, 0.6, 0.7,
                  0.8, 0.85, 0.86, 0.87, 0.88, 0.89,
                  0.9, 0.91, 0.92, 0.95, 0.98):
    metrics = check_metrics(df_dataset["val"]["labels"].values,
                            processed_preds,
                            processed_proba,
                            threshold)
    print(threshold, metrics)
    